In [2]:
import os
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'smiling-audio-287409-77f38c77b232.json'
from google.cloud import bigquery

In [3]:
# Create a Client object
client = bigquery.Client()

In [4]:
# Construct a reference to the "openaq" dataset
dataset_ref = client.dataset("openaq", project = "bigquery-public-data")

In [5]:
# API request - fetch the dataset
dataset = client.get_dataset(dataset_ref)

In [6]:
tables = list(client.list_tables(dataset))

In [7]:
for table in tables:
    print(table.table_id)

global_air_quality


In [9]:
# Construct a reference to the "global_air_quality" table
table_ref = dataset_ref.table("global_air_quality")

In [10]:
# API request - fetch the table
table = client.get_table(table_ref)

In [11]:
client.list_rows(table, max_results=6).to_dataframe()

C:\Users\user\anaconda4\envs\tensorflow gpu\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Cannot use bqstorage_client if max_results is set, reverting to fetching data with the tabledata.list endpoint.
  """Entry point for launching an IPython kernel.
C:\Users\user\anaconda4\envs\tensorflow gpu\lib\site-packages\ipykernel_launcher.py:1: PyarrowMissingWarning: Converting to a dataframe without pyarrow installed is often slower and will become unsupported in the future. Please install the pyarrow package.
  """Entry point for launching an IPython kernel.


,location,city,country,pollutant,value,timestamp,unit,source_name,latitude,longitude,averaged_over_in_hours
0,"BTM Layout, Bengaluru - KSPCB",Bengaluru,IN,co,910.00,2018-02-22 03:00:00+00:00,µg/m³,CPCB,12.912811,77.60922,0.25
1,"BTM Layout, Bengaluru - KSPCB",Bengaluru,IN,no2,131.87,2018-02-22 03:00:00+00:00,µg/m³,CPCB,12.912811,77.60922,0.25
2,"BTM Layout, Bengaluru - KSPCB",Bengaluru,IN,o3,15.57,2018-02-22 03:00:00+00:00,µg/m³,CPCB,12.912811,77.60922,0.25
3,"BTM Layout, Bengaluru - KSPCB",Bengaluru,IN,pm25,45.62,2018-02-22 03:00:00+00:00,µg/m³,CPCB,12.912811,77.60922,0.25
4,"BTM Layout, Bengaluru - KSPCB",Bengaluru,IN,so2,4.49,2018-02-22 03:00:00+00:00,µg/m³,CPCB,12.912811,77.60922,0.25
5,"BWSSB Kadabesanahalli, Bengaluru - KSPCB",Bengaluru,IN,co,840.00,2018-02-22 03:00:00+00:00,µg/m³,CPCB,12.938906,77.69727,0.25


In [15]:
first_query = """
        SELECT city
        FROM `bigquery-public-data.openaq.global_air_quality`
        WHERE country = 'US'
"""        
#If we use SELECT *, it returns all the columns satisfying the given condition      

In [17]:
#setting up the query with the query()

query_job = client.query(first_query)

In [18]:
#run the query and return pd df

us_cities = query_job.to_dataframe()

In [19]:
#what 5 cities have the most measurement

us_cities.city.value_counts().head()

Phoenix-Mesa-Scottsdale                     88
Houston                                     82
Los Angeles-Long Beach-Santa Ana            68
Riverside-San Bernardino-Ontario            60
New York-Northern New Jersey-Long Island    60
Name: city, dtype: int64

# Working With BigDatasets

In [23]:
second_query = """
        SELECT score, title
        FROM `bigquery-public-data.hacker_news.full`
        WHERE type = "job" 
        """

In [24]:
# creating the QueryJobConfig object to estimate the size of query without running it

dry_run_config = bigquery.QueryJobConfig(dry_run = True)

In [25]:
#API request - dry run query to estimate the cost

dry_run_query_job = client.query(second_query, job_config=dry_run_config)

In [26]:
print("This query will process {} bytes.".format(dry_run_query_job.total_bytes_processed))

#423MB

This query will process 423378625 bytes.


WE CAN ALSO LIMIT HOW MUCH DATA WE ARE WILLING TO SCAN

In [27]:
#only run the query if its less than 1MB

ONE_MB = 1000*1000
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed = ONE_MB)

In [28]:
#setup query (only run if its less than 1MB)

safe_query_job  = client.query(second_query, job_config=safe_config)

In [30]:
#API req 

safe_query_job.to_dataframe()


#ERROR- because its greater than the limit of 1MB we set

BadRequest: 400 GET https://bigquery.googleapis.com/bigquery/v2/projects/smiling-audio-287409/queries/6ba69c2c-abf6-4c4b-b92e-0fbc40f0ead4?maxResults=0&location=US: Query exceeded limit for bytes billed: 1000000. 423624704 or higher required.

(job ID: 6ba69c2c-abf6-4c4b-b92e-0fbc40f0ead4)

             -----Query Job SQL Follows-----             

    |    .    |    .    |    .    |    .    |    .    |
   1:
   2:        SELECT score, title
   3:        FROM `bigquery-public-data.hacker_news.full`
   4:        WHERE type = "job" 
   5:        
    |    .    |    .    |    .    |    .    |    .    |

In [33]:
ONE_GB = 1000*1000*1000
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed = ONE_GB)
safe_query_job  = client.query(second_query, job_config=safe_config)
job_post_scores=safe_query_job.to_dataframe()
job_post_scores

,score,title
0,4.0,Airbnb seeks UI designer
1,8.0,Nowmov (YC 2010W) - The Oracle
2,5.0,Backend Developer at Listia
3,2.0,Airbnb - Backend Engineer
4,5.0,Visual Designer for Mixpanel Analytics
...,...,...
13145,3.0,Software Engineer Intern at Mertado (YC W2010)
13146,3.0,Nowmov (YC 2010W) looking for Rails rockstars
13147,3.0,Nowmov - Part Time / Intern Designer
13148,3.0,Airbnb - Fraud Management Engineer


In [34]:
#posting average scores for job posts

job_post_scores.score.mean()

1.8809959430435128

In [43]:
#Which countries have reported pollution levels in units of "ppm"?
#In the code cell below, set first_query to an SQL query that pulls the appropriate entries from the country column.


third_query = """
            SELECT DISTINCT country
            FROM `bigquery-public-data.openaq.global_air_quality`
            WHERE unit = 'ppm'
"""

In [44]:
# Set up the query (cancel the query if it would use too much of 
# your quota, with the limit set to 10 GB)


safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
first_query_job = client.query(first_query, job_config=safe_config)


In [45]:

# API request - run the query, and return a pandas DataFrame
first_results = first_query_job.to_dataframe()


In [46]:
# View top few rows of results
print(first_results.head())

      city
0    BROWN
1    BROWN
2    BROWN
3    BROWN
4  Houston


In [47]:
#Which pollution levels were reported to be exactly 0?

#Set zero_pollution_query to select all columns of the rows where the value column is 0.
#Set zero_pollution_results to a pandas DataFrame containing the query results.


# Query to select all columns where pollution levels are exactly 0
zero_pollution_query = """
                    SELECT *
                    FROM `bigquery-public-data.openaq.global_air_quality`
                    WHERE value = 0
                   

"""
# Your code goes here

# Set up the query
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
query_job = client.query(zero_pollution_query, job_config=safe_config)

# API request - run the query and return a pandas DataFrame
zero_pollution_results = query_job.to_dataframe() # Your code goes here

print(zero_pollution_results.head())

                                        location     city country pollutant  \
0                     Victoria Memorial - WBSPCB  Kolkata      IN      pm25   
1  Rabindra Bharati University, Kolkata - WBSPCB  Kolkata      IN       so2   
2                   Zamość ul. Hrubieszowska 69A   Zamość      PL       no2   
3                               Końskie, MOBILNA  Końskie      PL      pm10   
4                               Końskie, MOBILNA  Końskie      PL      pm25   

   value                 timestamp   unit source_name   latitude  longitude  \
0    0.0 2017-10-16 20:45:00+00:00  µg/m³        CPCB  22.572645  88.363890   
1    0.0 2017-10-28 14:30:00+00:00  µg/m³        CPCB  22.627874  88.380400   
2    0.0 2020-05-19 05:00:00+00:00  µg/m³        GIOS  50.716630  23.290247   
3    0.0 2018-12-21 13:00:00+00:00  µg/m³        GIOS  51.189526  20.408892   
4    0.0 2018-12-21 13:00:00+00:00  µg/m³        GIOS  51.189526  20.408892   

   averaged_over_in_hours  
0                    0